# Chapter 2 — An Array of Sequences

**Sections with code snippets in this chapter:**

* [List Comprehensions and Generator Expressions](#List-Comprehensions-and-Generator-Expressions)
* [Tuples Are Not Just Immutable Lists](#Tuples-Are-Not-Just-Immutable-Lists)
* [Unpacking sequences and iterables](#Unpacking-sequences-and-iterables)
* [Pattern Matching with Sequences](#Pattern-Matching-with-Sequences)
* [Slicing](#Slicing)
* [Using + and * with Sequences](#Using-+-and-*-with-Sequences)
* [Augmented Assignment with Sequences](#Augmented-Assignment-with-Sequences)
* [list.sort and the sorted Built-In Function](#list.sort-and-the-sorted-Built-In-Function)
* [When a List Is Not the Answer](#When-a-List-Is-Not-the-Answer)
* [Memory Views](#Memory-Views)
* [NumPy and SciPy](#NumPy-and-SciPy)
* [Deques and Other Queues](#Deques-and-Other-Queues)
* [Soapbox](#Soapbox)

## List Comprehensions and Generator Expressions

#### Example 2-1. Building a list of Unicode codepoints from a string

In [1]:
symbols = '$¢£¥€¤'
codes = []

for symbol in symbols:
    codes.append(ord(symbol))

codes

[36, 162, 163, 165, 8364, 164]

#### Example 2-2. Build a list of Unicode codepoints from a string, using a listcomp

In [3]:
symbols = '$¢£¥€¤'

codes = [ord(symbol) for symbol in symbols]

codes

[36, 162, 163, 165, 8364, 164]

#### Box: Listcomps No Longer Leak Their Variables

In [4]:
x = 'ABC'
codes = [ord(x) for x in x]
x

'ABC'

In [5]:
codes

[65, 66, 67]

In [6]:
codes = [last := ord(c) for c in x]
last

67

#### Example 2-3. The same list built by a listcomp and a map/filter composition

In [8]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

[162, 163, 165, 8364, 164]

In [9]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]